# QC Data and Filter Samples for DE testing

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scanpy as sc
import seaborn as sns
from IPython.display import HTML
import html
from tqdm import tqdm

from pydeseq2.ds import DeseqStats
from pydeseq2.dds import DeseqDataSet



In [ ]:
NUM_CPUS = 8
# DATA_PATH = os.getcwd()
# DATA_PATH = '/data/expression_atlas/v1/GSE139358/'
# DATA_PATH = '/data/expression_atlas/v1/GSE112087/'
# DATA_PATH = '/data/expression_atlas/v1/GSE122459/'
# DATA_PATH = '/data/expression_atlas/v1/GSE102371'
# DATA_PATH = '/data/expression_atlas/v1/GSE110914/'
# DATA_PATH = '/data/expression_atlas/v1/GSE162828/'
# DATA_PATH = '/data/expression_atlas/v1/GSE102371/'
DATA_PATH = '/data/expression_atlas/v1/GSE80183/'


MULTIQC_PATH = os.path.join(DATA_PATH, 'rnaseq_output/multiqc/star_salmon/multiqc_report.html')

COUNT_PATH = os.path.join(DATA_PATH, 'rnaseq_output/star_salmon')
METADATA_FH = '' + '%s_metadata.csv' % DATA_PATH.rstrip('/').split('/')[-1]

RESULTS_PATH = '' + 'results/%s' % DATA_PATH.rstrip('/').split('/')[-1]

if not os.path.exists('results'):
    os.mkdir('results')
DDS_TRANSCRIPT_FH = '' + 'results/%s_dds_transcript.h5_ad' % DATA_PATH.rstrip('/').split('/')[-1]
DDS_GENE_FH = '' + 'results/%s_dds_gene.h5_ad' % DATA_PATH.rstrip('/').split('/')[-1]

TRANSCRIPT_SUM_FILTER = 10
TRANSCRIPT_PSEUDOCOUNT = 0
GENE_SUM_FILTER = 10
GENE_PSEUDOCOUNT = 0

In [ ]:
# Embed the MulitQC report into notebook. Note srcdoc was reqiured to get the html rendered without screwing up the 
# styling of the notebook while using an iframe. Buttons on the side don't work, but everything else seems to work fine.

with open(MULTIQC_PATH,'r') as f_in:
    html_raw = html.escape(f_in.read())

HTML('<iframe srcdoc="%s" width="1200px" height="1000px"></iframe>' % html_raw)

In [ ]:
# Read sample metadata into dataframe

metadata = pd.read_csv(os.path.join('', METADATA_FH), index_col=0)
smallest_condition_size = metadata[[c for c in metadata.columns if c.startswith('condition')]].value_counts()[-1]

metadata, smallest_condition_size

In [ ]:
# Merge effective lengths and average across all runs for transcript and gene dataframes.

# Build the transcript length dataframe off of the first sample in the metadata dataframe.
transcript_length = pd.read_csv(os.path.join(COUNT_PATH, metadata.index[0], 'quant.sf'), delimiter= '\t', index_col=0)
transcript_length.rename({'EffectiveLength':'EffectiveLength_%s' % metadata.index[0]}, axis=1, inplace=True)
transcript_length.drop(['Length', 'TPM', 'NumReads'], inplace=True, axis=1)

gene_length = pd.read_csv(os.path.join(COUNT_PATH, metadata.index[0], 'quant.genes.sf'), delimiter= '\t', index_col=0)
gene_length.rename({'EffectiveLength':'EffectiveLength_%s' % metadata.index[0]}, axis=1, inplace=True)
gene_length.drop(['Length', 'TPM', 'NumReads'], inplace=True, axis=1)

# Populate samples into effective length dataframe with remaining samples.
for srx in tqdm(metadata.index[1:]):
    df = pd.read_csv(os.path.join(COUNT_PATH, srx, 'quant.sf'), delimiter='\t', index_col=0)
    df.drop(['Length', 'TPM', 'NumReads'], inplace=True, axis=1)
    df.rename({'EffectiveLength':'EffectiveLength_%s' % srx}, axis=1, inplace=True)
    transcript_length = transcript_length.merge(df, on='Name')

    df_gene = pd.read_csv(os.path.join(COUNT_PATH, srx, 'quant.genes.sf'), delimiter='\t', index_col=0)
    df_gene.drop(['Length', 'TPM', 'NumReads'], inplace=True, axis=1)
    df_gene.rename({'EffectiveLength':'EffectiveLength_%s' % srx}, axis=1, inplace=True)
    gene_length = gene_length.merge(df_gene, on='Name')


# Average effective lengths. It would probably be better to use a weighted averaging similar to tximport.
transcript_length['length'] = transcript_length.mean(axis=1)
gene_length['length'] = gene_length.mean(axis=1)
transcript_length.drop([c for c in transcript_length.columns if c.startswith('EffectiveLength_')], inplace=True, axis=1)
gene_length.drop([c for c in gene_length.columns if c.startswith('EffectiveLength_')], inplace=True, axis=1)

transcript_length.shape, gene_length.shape

In [ ]:
# Read in transcript and gene count/TPM dataframes.

expression = pd.read_csv(os.path.join(COUNT_PATH, 'salmon.merged.transcript_counts.tsv'), delimiter = '\t', index_col=0)
gene_transcript_mapping = expression[['gene_id']].copy().reset_index()
expression.drop('gene_id', inplace=True, axis=1)

tpm = pd.read_csv(os.path.join(COUNT_PATH, 'salmon.merged.transcript_tpm.tsv'), delimiter = '\t', index_col=0)
tpm.drop('gene_id', inplace=True, axis=1)

# See: https://nf-co.re/rnaseq/3.12.0/docs/output#salmon on output choice below 
# salmon.merged.gene_counts_length_scaled.tsv is the gene-level output of nf-core rnaseq that is bias-corrected
# and is already scaled by potential transcript length
expression_gene = pd.read_csv(os.path.join(COUNT_PATH, 'salmon.merged.gene_counts_length_scaled.tsv'), delimiter='\t', index_col=0)
expression_gene.drop('gene_name', inplace=True, axis=1)

tpm_gene = pd.read_csv(os.path.join(COUNT_PATH, 'salmon.merged.gene_tpm.tsv'), delimiter='\t', index_col=0)
tpm_gene.drop('gene_name', inplace=True, axis=1)

expression.shape, expression_gene.shape


In [ ]:
# Filter expression dataframes on samples not in metadata.csv

samples_to_drop = [c for c in expression.columns if c not in metadata.index]

expression.drop(samples_to_drop, axis=1, inplace=True)
expression_gene.drop(samples_to_drop, axis=1, inplace=True)
tpm.drop(samples_to_drop, axis=1, inplace=True)
tpm_gene.drop(samples_to_drop, axis=1, inplace=True)
expression

In [ ]:
# Drop specific samples from dataframe. Provide accession name of samples to remove from analysis.

samples_to_drop = []
# samples_to_drop = ['SRX3729696']
# samples_to_drop = ['SRX9647190']
expression.drop(samples_to_drop, axis=1, inplace=True)
expression_gene.drop(samples_to_drop, axis=1, inplace=True)
tpm.drop(samples_to_drop, axis=1, inplace=True)
tpm_gene.drop(samples_to_drop, axis=1, inplace=True)
metadata.drop(samples_to_drop, axis=1, inplace=True)
expression

In [ ]:
# Drop specific conditions/groups from metadata dataframe. 

groups_to_drop = []
metadata.drop(groups_to_drop, axis=1, inplace=True)
metadata

In [ ]:
# Filter expression dataframe and prepare for QC/EDA.

filtered_expression_transcript = expression.T.copy()
filtered_expression_transcript = filtered_expression_transcript[
                                    filtered_expression_transcript.columns[
                                        filtered_expression_transcript.sum(axis=0) >= TRANSCRIPT_SUM_FILTER
                                        ]
                                    ]

# Tag to take another look, filter later 
filtered_expression_transcript = filtered_expression_transcript[
                                    filtered_expression_transcript.columns[
                                        (filtered_expression_transcript >= TRANSCRIPT_SUM_FILTER).sum(axis=0) >
                                            smallest_condition_size
                                        ]
                                    ]

filtered_tpm_transcript = tpm.T.copy()
filtered_tpm_transcript = filtered_tpm_transcript[filtered_expression_transcript.columns]

filtered_expression_gene = expression_gene.T.copy()
filtered_expression_gene = filtered_expression_gene[
                                    filtered_expression_gene.columns[
                                        filtered_expression_gene.sum(axis=0) >= GENE_SUM_FILTER
                                        ]
                                    ]

# Tag to take another look, filter later 
filtered_expression_gene = filtered_expression_gene[
                                    filtered_expression_gene.columns[
                                        (filtered_expression_gene >= GENE_SUM_FILTER).sum(axis=0) > 
                                            smallest_condition_size
                                        ]
                                    ]

filtered_tpm_gene = tpm_gene.T.copy()
filtered_tpm_gene = filtered_tpm_gene[filtered_expression_gene.columns]

assert (
        all([i == j for i,j in zip(filtered_expression_gene.columns, filtered_tpm_gene.columns)]) and 
        all([i == j for i,j in zip(filtered_expression_transcript.columns, filtered_tpm_transcript.columns)]) and 
        all([i == j for i,j in zip(filtered_expression_gene.index, filtered_tpm_gene.index)]) and
        all([i == j for i,j in zip(filtered_expression_transcript.index, filtered_tpm_transcript.index)])
    )

(   expression.shape, 
    filtered_expression_transcript.shape, 
    expression_gene.shape, 
    filtered_expression_gene.shape, 
    filtered_tpm_transcript.shape, 
    filtered_tpm_gene.shape, 
    )


In [ ]:
# Create a Deseq dataframe (AnnData object).

# DeseqDataSet expects integers in counts matrix, need to check in on the default method for 
# rounding fractional counts to integers in tximport.
 
dds = DeseqDataSet(
    counts = filtered_expression_transcript.astype(int), 
    metadata = metadata, 
    design_factors = 
        [c for c in metadata.columns if c.startswith('group')]+
        [c for c in metadata.columns if c.startswith('condition')],
    refit_cooks = True, 
    n_cpus = NUM_CPUS, 
    )

dds_gene = DeseqDataSet(
    counts = filtered_expression_gene.astype(int), 
    metadata = metadata, 
    design_factors = 
        [c for c in metadata.columns if c.startswith('group')]+
        [c for c in metadata.columns if c.startswith('condition')],
    refit_cooks = True, 
    n_cpus = NUM_CPUS, 
    )

In [ ]:
# Set gene-transcript mapping attribute in uns for comparisons between 
# gene- and transcript-level quantifications.

dds.uns['gene_transcript_mapping'] = gene_transcript_mapping
dds_gene.uns['gene_transcript_mapping'] = gene_transcript_mapping

In [ ]:
# Set raw TPMs as layer in dds objects.

dds.layers['raw_tpm'] = np.array(filtered_tpm_transcript)
dds_gene.layers['raw_tpm'] = np.array(filtered_tpm_gene)

In [ ]:
# Merge the average effective lengths into the var dataframe.

dds.var = dds.var.merge(transcript_length, left_index=True, right_index=True)
dds_gene.var = dds_gene.var.merge(gene_length, left_index=True, right_index=True)

In [ ]:
# Compute size-factors and library sizes.

dds.fit_size_factors()
dds.obs['size_factors'] = dds.obsm['size_factors']
dds.obs['lib_sizes'] = dds.X.sum(axis=1)

dds_gene.fit_size_factors()
dds_gene.obs['size_factors'] = dds_gene.obsm['size_factors']
dds_gene.obs['lib_sizes'] = dds_gene.X.sum(axis=1)

dds.obs

In [ ]:
# Variance-stabilizing transformation.

dds.vst()
dds_gene.vst()

dds.layers['vst_counts'], dds_gene.layers['vst_counts']

In [ ]:
# Set recoverable count data.

dds.layers['counts'] = dds.X.copy()
dds_gene.layers['counts'] = dds_gene.X.copy()

In [ ]:
# Compute fractional counts to get a quick idea for any weird skews in library composition.

dds.layers['fraction_counts'] = dds.layers['counts'] / np.reshape(dds.layers['counts'].sum(axis=1), (-1,1))
dds_gene.layers['fraction_counts'] = dds_gene.layers['counts'] / np.reshape(dds_gene.layers['counts'].sum(axis=1), (-1,1))

dds.layers['fraction_counts'], dds_gene.layers['fraction_counts']


In [ ]:
# Plot CDF of fractional composition of libraries. 

ax_transcript = sns.ecdfplot(np.log2(dds.layers['fraction_counts'].T))
ax_transcript.set_xlabel('log2 fraction counts')
ax_transcript.legend(
        labels=dds.obs.index, 
        loc='upper left', 
        bbox_to_anchor=(1.,1.), 
        ncols=1 if len(dds.obs.index) < 10 else int(len(dds.obs.index)/10),
        frameon=False,
    )
ax_transcript.set_title('transcript')
plt.show()

ax_gene = sns.ecdfplot(np.log2(dds_gene.layers['fraction_counts'].T))
ax_gene.set_xlabel('log2 fraction counts')
ax_gene.legend(
        labels=dds_gene.obs.index, 
        loc='upper left', 
        bbox_to_anchor=(1.,1.), 
        ncols=1 if len(dds_gene.obs.index) < 10 else int(len(dds_gene.obs.index)/10),
        frameon=False,
    )
ax_gene.set_title('gene')
plt.show()


In [ ]:
# Replace count matrix with variance-transformed counts, following DESeq2 recommendation
# for preprocessing count data before QC visualization.

dds.X = dds.layers['vst_counts'].copy()
dds_gene.X = dds_gene.layers['vst_counts'].copy()

np.nan_to_num(dds.X, copy=False)
np.nan_to_num(dds_gene.X, copy=False)

dds.layers['counts'], dds.X, dds.layers['vst_counts']

In [ ]:
# Scale transformed variables.

sc.pp.scale(dds)
sc.pp.scale(dds_gene)

np.nan_to_num(dds.X, copy=False)
np.nan_to_num(dds_gene.X, copy=False)

dds.X.mean(axis=0), dds.X.std(axis=0), dds_gene.X.mean(axis=0), dds_gene.X.std(axis=0)

In [ ]:
# Preliminary PCA on transcript- and gene-level data.

suffix_size = 4

sc.pp.pca(dds)
ax_transcript_pca = sc.pl.pca( 
    dds, 
    color=
        [c for c in dds.obs.columns if c.startswith('group')]+
        [c for c in dds.obs.columns if c.startswith('condition')]+
        ['lib_sizes'], 
    size = 128,
    show=False,
    )

for i, s in enumerate(dds.obsm['X_pca']):
    if type(ax_transcript_pca) == list:
        for ax in ax_transcript_pca:
            ax.text(s[0], s[1], dds.obs.index[i][-suffix_size:])
    else:
        ax_transcript_pca.text(s[0], s[1], dds.obs.index[i][-suffix_size:])

sc.pp.pca(dds_gene)
ax_gene_pca = sc.pl.pca(
    dds_gene, 
    color=
        [c for c in dds_gene.obs.columns if c.startswith('group')]+
        [c for c in dds_gene.obs.columns if c.startswith('condition')]+
        ['lib_sizes'],
    size = 128,
    show=False, 
    )

for i, s in enumerate(dds_gene.obsm['X_pca']):
    if type(ax_gene_pca) == list:
        for ax in ax_gene_pca:
            ax.text(s[0], s[1], dds_gene.obs.index[i][-suffix_size:])
    else:
        ax_gene_pca.text(s[0], s[1], dds_gene.obs.index[i][-suffix_size:])


In [ ]:
# Plot explained variance ratios.

fig, ax = plt.subplots(1,2,figsize=(10,5))

ax[0].plot(dds.uns['pca']['variance_ratio'])
ax[1].plot(dds_gene.uns['pca']['variance_ratio'])

ax[0].set_ylabel('fraction explained variance')
ax[0].set_xlabel('PC')
ax[1].set_xlabel('PC')
ax[0].set_title('Transcript')
ax[1].set_title('Gene')


In [ ]:
# Plot loadings for first 3 PCs.

sc.pl.pca_loadings(dds, components = '1,2,3')
sc.pl.pca_loadings(dds_gene, components = '1,2,3')

In [ ]:
# Sample-sample pearson correlation.

dds.layers['vst_counts'].shape
dds_gene.layers['vst_counts'].shape

dist = np.corrcoef(np.nan_to_num(dds.layers['vst_counts'] / 
                                 np.reshape(dds.obs['lib_sizes'], (-1, 1)), copy=False))
ax_transcript = sns.heatmap(
                dist, 
                xticklabels=dds.obs.index, 
                yticklabels=dds.obs['condition-1'], 
                cbar_kws={'label': 'pearson r'}, 
            )
ax_transcript.set_title('transcript')
plt.show()

dist = np.corrcoef(np.nan_to_num(dds_gene.layers['vst_counts'] / 
                                 np.reshape(dds_gene.obs['lib_sizes'], (-1, 1)), copy=False))
ax_gene = sns.heatmap(
                dist, 
                xticklabels=dds_gene.obs.index, 
                yticklabels=dds_gene.obs['condition-1'], 
                cbar_kws={'label': 'pearson r'},
            )
ax_gene.set_title('gene')
plt.show()



In [ ]:
# Restore the original counts data.

dds.X = dds.layers['counts'].copy()
dds_gene.X = dds_gene.layers['counts'].copy()


In [ ]:
# Fit dispersions, logFCs, and calculate cooks.

dds.deseq2()
dds_gene.deseq2()

In [ ]:
# Plot fitted dispersions.

fig, ax = plt.subplots(1,2,figsize=(10,5))

ax[0].scatter(
        np.log(dds.varm['_normed_means']), 
        np.log(dds.varm['genewise_dispersions']), 
        s=1, 
        alpha=0.01, 
        label='raw',
    )
ax[0].scatter(
        np.log(dds.varm['_normed_means']), 
        np.log(dds.varm['dispersions']), 
        s=1, 
        alpha=0.01, 
        label='squeezed',
    )
ax[0].scatter(
        np.log(dds.varm['_normed_means']), 
        np.log(dds.varm['fitted_dispersions']), 
        s=1, 
        alpha=0.01, 
        label='trended', 
        c='r', 
    )
ax[0].set_ylabel('log dispersions')
ax[0].set_xlabel('log normalized mean')
ax[0].set_title('transcript-level')
ax[0].legend(frameon=False)
legend = ax[0].legend(frameon=False)
for lh in legend.legend_handles:
    lh.set_alpha(1)

ax[1].scatter(
        np.log(dds_gene.varm['_normed_means']), 
        np.log(dds_gene.varm['genewise_dispersions']), 
        s=1, 
        alpha=0.01, 
        label='raw',
    )
ax[1].scatter(
        np.log(dds_gene.varm['_normed_means']), 
        np.log(dds_gene.varm['dispersions']), 
        s=1, 
        alpha=0.01, 
        label='squeezed',
    )
ax[1].scatter(
        np.log(dds_gene.varm['_normed_means']), 
        np.log(dds_gene.varm['fitted_dispersions']), 
        s=1, 
        alpha=0.01, 
        label='trended', 
        c='r', 
    )
ax[1].set_xlabel('log normalized mean')
ax[1].set_title('gene-level')
legend = ax[1].legend(frameon=False)
for lh in legend.legend_handles:
    lh.set_alpha(1)

In [ ]:
# Manually define contrasts given conditions in metadata dataframe.

for c in [c for c in dds.obs.columns if c.startswith('condition')]:
    print(dds.obs[c].unique())

# Pydeseq2 contrasts require condition-name, treatment level, reference level format.
# contrasts = {
#     'SLE_v_control': ['condition-1','TREAT-1','CONTROL'],
#     }

contrasts = {
    'SLE_v_control': ['condition-1','DISEASE','CONTROL'],
    }

# contrasts = {
#     'T1D_v_control': ['condition-1','TREAT-1','CONTROL'],
#     'preT1D_v_control': ['condition-1','TREAT-2','CONTROL']
#     }

# contrasts = {
#     'SLE_v_control': ['condition-1','TREAT-1','CONTROL']
#     }

# contrasts = {
#     'T1D_v_control': ['condition-1','TREAT-1','CONTROL'],
#     }

In [ ]:
dds.obsm['design_matrix'], dds_gene.obsm['design_matrix']

In [ ]:
# Create Stats object. Define relevant contrasts for DE and LogFC computations and run tests. 

# Holds all DeseqStats objects as defined in contrasts.
dds.uns['stat_results'] = {}
dds_gene.uns['stat_results'] = {}

for k, v in contrasts.items():

    stat_res = DeseqStats(dds, contrast=v, n_cpus=NUM_CPUS)
    stat_res_gene = DeseqStats(dds_gene, contrast=v, n_cpus=NUM_CPUS)

    stat_res.summary()
    stat_res_gene.summary()

    stat_res.lfc_shrink()
    stat_res_gene.lfc_shrink()
    
    dds.uns['stat_results'][k] = stat_res.results_df
    dds_gene.uns['stat_results'][k] = stat_res_gene.results_df

In [ ]:
# Calculate -log10_padj and fill NaN. NaN caused by filtering genes with cooks outliers.

for i, k in enumerate(contrasts.keys()):
    
    dds.uns['stat_results'][k]['-log10_padj'] = -1. * np.log10(dds.uns['stat_results'][k]['padj'])
    dds_gene.uns['stat_results'][k]['-log10_padj'] = -1. * np.log10(dds_gene.uns['stat_results'][k]['padj'])

    dds.uns['stat_results'][k]['-log10_padj'].fillna(0.0, inplace=True)
    dds_gene.uns['stat_results'][k]['-log10_padj'].fillna(0.0, inplace=True)

    dds.uns['stat_results'][k]['-log10_padj'].replace(np.inf, 30., inplace=True)
    dds_gene.uns['stat_results'][k]['-log10_padj'].replace(np.inf, 30., inplace=True)


In [ ]:
# Dump results dataframes to results folder.

for k in contrasts.keys():

    dds.uns['stat_results'][k].to_csv('%s_%s_%s.csv' % (RESULTS_PATH, 'transcript', k))
    dds_gene.uns['stat_results'][k].to_csv('%s_%s_%s.csv' % (RESULTS_PATH, 'gene', k))


In [ ]:
# Write dds objects to files for DE and LogFC calculations.

# Pydeseq2 supports trend_coeffs/replaced as either np.array or pd.series, np.array required for 
# saving h5-formatted AnnData objects.
dds.uns['trend_coeffs'] = np.array(dds.uns['trend_coeffs'])
dds_gene.uns['trend_coeffs'] = np.array(dds_gene.uns['trend_coeffs'])

dds.varm['replaced'] = np.array(dds.varm['replaced'])
dds_gene.varm['replaced'] = np.array(dds_gene.varm['replaced'])

# Add contrasts to uns.
dds.uns['contrasts'] = contrasts
dds_gene.uns['contrasts'] = contrasts

# DeseqDataSet doesn't have native support for writing h5, save as AnnData objects and restore from
# AnnData objects.

dds.write(DDS_TRANSCRIPT_FH)
dds_gene.write(DDS_GENE_FH)